In [1]:
import pandas as pd

# CSV file load karo
df = pd.read_csv("CloudWatch_Traffic_Web_Attack.csv")


In [2]:
from collections import defaultdict

# For counting requests from IPs
ip_request_count = defaultdict(int)
historical_avg_traffic = defaultdict(lambda: 10000)  # Example: fill dummy average
traffic_today = defaultdict(int)
ports_accessed_today = defaultdict(set)
baseline_port_count = defaultdict(lambda: 3)

# Sample threat IP list
threat_intel_list = ['192.168.1.1', '185.13.44.23']

# Known ports
known_ports = [80, 443, 21, 22, 25]


In [5]:
for idx, row in df.iterrows():
    flag = None

    # Rule 1: Frequency spike
    ip_request_count[row['src_ip']] += 1
    if ip_request_count[row['src_ip']] > 5 * 1:  # Assume avg = 1
        flag = "Bhai: Is IP ki request frequency bahut badh gayi h"

    # Rule 2: Impossible Travel Rule
    # Find another row with same src_ip but different time
    other_rows = df[(df['src_ip'] == row['src_ip']) & (df['time'] != row['time'])]
    other_row = other_rows.iloc[0] if not other_rows.empty else None
    if other_row is not None:
        time_diff = abs((pd.to_datetime(row['time']) - pd.to_datetime(other_row['time'])).total_seconds())
        if (row['src_ip'] == other_row['src_ip']) and (row['src_ip_country_code'] != other_row['src_ip_country_code']) and (time_diff < 30):
            flag = "Bhai: Ek hi IP alag-alag country se short time me access kar rahi h"

    # Rule 3: Access During Holiday or Weekend
    if pd.to_datetime(row['time']).weekday() in [5, 6]:  # 5=Saturday, 6=Sunday
        flag = "Bhai: Weekend ya holiday par access hua h"

    # Rule 4: Port Count Anomaly
    ports_accessed_today[row['src_ip']].add(row['dst_port'])
    if len(ports_accessed_today[row['src_ip']]) > baseline_port_count[row['src_ip']]:
        flag = "Bhai: Is IP ne jyada ports access kiye h"

    # Rule 5: IP reputation
    if row['src_ip'] in threat_intel_list:
        flag = "Bhai: IP malicious list me mila h"

    # Rule 6: Unauthorized Sensitive File Access
    if row['source.name'] == "prod_webserver" and row['src_ip_country_code'] != "IN":
        flag = "Bhai: Foreign IP ne prod server access kiya h"

    # Rule 7: Abnormal Payload Size in GET Request
    if row['rule_names'] == "GET Request" and row['bytes_out'] > 1000000:
        flag = "Bhai: GET request me payload bahut zyada h"

    # Rule 8: Abnormal Payload Size in POST Request
    if row['rule_names'] == "POST Request" and row['bytes_out'] > 500000:
        flag = "Bhai: POST request me payload bahut zyada h"

    # Rule 9: File Download at Unusual Time
    hour = pd.to_datetime(row['time']).hour
    if hour in [2, 3, 4]:
        flag = "Bhai: Raat ke time suspicious activity hui h"

    # Rule 10: Unknown port
    if row['dst_port'] not in known_ports:
        flag = "Bhai: Unknown port access hua h"

    if flag:
        print(f"{row['src_ip']} → {flag}")


147.161.161.82 → Bhai: Foreign IP ne prod server access kiya h
165.225.33.6 → Bhai: Foreign IP ne prod server access kiya h
165.225.212.255 → Bhai: Foreign IP ne prod server access kiya h
136.226.64.114 → Bhai: Foreign IP ne prod server access kiya h
165.225.240.79 → Bhai: Foreign IP ne prod server access kiya h
136.226.77.103 → Bhai: Foreign IP ne prod server access kiya h
165.225.26.101 → Bhai: Foreign IP ne prod server access kiya h
155.91.45.242 → Bhai: Foreign IP ne prod server access kiya h
165.225.209.4 → Bhai: Foreign IP ne prod server access kiya h
147.161.131.1 → Bhai: Foreign IP ne prod server access kiya h
165.225.33.6 → Bhai: Foreign IP ne prod server access kiya h
136.226.67.101 → Bhai: Foreign IP ne prod server access kiya h
165.225.212.255 → Bhai: Foreign IP ne prod server access kiya h
136.226.64.114 → Bhai: Foreign IP ne prod server access kiya h
94.188.248.74 → Bhai: Foreign IP ne prod server access kiya h
165.225.240.79 → Bhai: Foreign IP ne prod server access kiya 